# AI Agents with Amazon Nova - Practical Use Cases

This notebook walks you through creating AI agents for different real-world use cases using Amazon Nova API.

## What We'll Cover:
1. **Customer Support Agent** - Handles customer inquiries with knowledge base
2. **Research Assistant Agent** - Searches web and summarizes information
3. **Data Analysis Agent** - Analyzes data and generates insights
4. **Task Automation Agent** - Automates workflows with multiple tools
5. **Multi-Agent System** - Coordinating multiple specialized agents

Let's get started!

## Setup and Configuration

In [ ]:
# Import required libraries
from openai import OpenAI
import os
from dotenv import load_dotenv
import json
from datetime import datetime

# Load environment variables
load_dotenv()

# Initialize Nova API client
api_key = os.getenv("NOVA_API_KEY")
base_url = "https://api.nova.amazon.com/v1"
client = OpenAI(api_key=api_key, base_url=base_url)

# Choose your model
model_id = 'nova-2-lite-v1'  # Fast and cost-effective
# model_id = 'nova-2-pro-v1'  # More capable for complex tasks

print(f"✅ Connected to Amazon Nova API")
print(f"📦 Using model: {model_id}")

## Use Case 1: Customer Support Agent

This agent handles customer inquiries by:
- Accessing a knowledge base
- Checking order status
- Providing product information
- Escalating complex issues

In [ ]:
# Define tools for customer support agent
customer_support_tools = [
    {
        "type": "function",
        "function": {
            "name": "search_knowledge_base",
            "description": "Search the company knowledge base for product information, policies, and FAQs",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The search query"
                    },
                    "category": {
                        "type": "string",
                        "enum": ["products", "policies", "troubleshooting", "billing"],
                        "description": "Category to search in"
                    }
                },
                "required": ["query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "check_order_status",
            "description": "Check the status of a customer order",
            "parameters": {
                "type": "object",
                "properties": {
                    "order_id": {
                        "type": "string",
                        "description": "The order ID to check"
                    }
                },
                "required": ["order_id"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "create_support_ticket",
            "description": "Create a support ticket for complex issues that need human attention",
            "parameters": {
                "type": "object",
                "properties": {
                    "priority": {
                        "type": "string",
                        "enum": ["low", "medium", "high", "urgent"],
                        "description": "Priority level"
                    },
                    "summary": {
                        "type": "string",
                        "description": "Brief summary of the issue"
                    },
                    "details": {
                        "type": "string",
                        "description": "Detailed description"
                    }
                },
                "required": ["priority", "summary", "details"]
            }
        }
    }
]

print("✅ Customer Support Tools defined")

In [ ]:
# Simulated tool implementations (in production, these would call real APIs)
def search_knowledge_base(query, category=None):
    """Simulate knowledge base search"""
    kb = {
        "return policy": "We offer 30-day returns on all products. Items must be unused and in original packaging.",
        "shipping time": "Standard shipping takes 5-7 business days. Express shipping is 2-3 days.",
        "warranty": "All products come with a 1-year manufacturer warranty."
    }
    for key, value in kb.items():
        if query.lower() in key:
            return {"result": value, "category": category or "general"}
    return {"result": "No information found. Please contact support.", "category": category or "general"}

def check_order_status(order_id):
    """Simulate order status check"""
    return {
        "order_id": order_id,
        "status": "In Transit",
        "estimated_delivery": "2026-01-05",
        "tracking_number": "TRK123456789"
    }

def create_support_ticket(priority, summary, details):
    """Simulate ticket creation"""
    ticket_id = f"TICKET-{datetime.now().strftime('%Y%m%d%H%M%S')}"
    return {
        "ticket_id": ticket_id,
        "priority": priority,
        "status": "Created",
        "message": "Support ticket created successfully. A team member will contact you within 24 hours."
    }

# Tool dispatcher
def execute_tool(tool_name, tool_args):
    """Execute the appropriate tool based on name"""
    tools = {
        "search_knowledge_base": search_knowledge_base,
        "check_order_status": check_order_status,
        "create_support_ticket": create_support_ticket
    }
    if tool_name in tools:
        return tools[tool_name](**tool_args)
    return {"error": f"Unknown tool: {tool_name}"}

print("✅ Tool implementations ready")

In [ ]:
# Customer Support Agent Implementation
def run_customer_support_agent(user_message):
    """Run the customer support agent with tool calling"""
    
    messages = [
        {
            "role": "system",
            "content": """You are a helpful customer support agent. Your goal is to:
            1. Understand customer inquiries
            2. Use available tools to find information
            3. Provide clear, friendly responses
            4. Escalate complex issues when needed
            
            Always be polite, professional, and empathetic."""
        },
        {
            "role": "user",
            "content": user_message
        }
    ]
    
    print(f"\n🤖 Customer Support Agent")
    print(f"📝 User: {user_message}\n")
    
    # Initial API call
    response = client.chat.completions.create(
        model=model_id,
        messages=messages,
        tools=customer_support_tools,
        tool_choice="auto"
    )
    
    assistant_message = response.choices[0].message
    
    # Handle tool calls
    if assistant_message.tool_calls:
        messages.append(assistant_message)
        
        for tool_call in assistant_message.tool_calls:
            tool_name = tool_call.function.name
            tool_args = json.loads(tool_call.function.arguments)
            
            print(f"🔧 Calling tool: {tool_name}")
            print(f"   Arguments: {tool_args}")
            
            # Execute tool
            tool_result = execute_tool(tool_name, tool_args)
            print(f"   Result: {tool_result}\n")
            
            # Add tool result to messages
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": json.dumps(tool_result)
            })
        
        # Get final response
        final_response = client.chat.completions.create(
            model=model_id,
            messages=messages
        )
        
        final_message = final_response.choices[0].message.content
    else:
        final_message = assistant_message.content
    
    print(f"💬 Agent: {final_message}")
    return final_message

# Test the customer support agent
run_customer_support_agent("What's your return policy?")

In [ ]:
# Test with order tracking
run_customer_support_agent("Can you check the status of order ORD-12345?")

## Use Case 2: Research Assistant Agent

This agent helps with research by:
- Searching for information
- Summarizing findings
- Comparing multiple sources
- Generating research reports

In [ ]:
# Define tools for research assistant
research_tools = [
    {
        "type": "function",
        "function": {
            "name": "web_search",
            "description": "Search the web for information on a topic",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The search query"
                    },
                    "num_results": {
                        "type": "integer",
                        "description": "Number of results to return",
                        "default": 5
                    }
                },
                "required": ["query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "save_research_note",
            "description": "Save important findings to research notes",
            "parameters": {
                "type": "object",
                "properties": {
                    "title": {
                        "type": "string",
                        "description": "Title of the note"
                    },
                    "content": {
                        "type": "string",
                        "description": "Content to save"
                    },
                    "tags": {
                        "type": "array",
                        "items": {"type": "string"},
                        "description": "Tags for categorization"
                    }
                },
                "required": ["title", "content"]
            }
        }
    }
]

# Tool implementations
def web_search(query, num_results=5):
    """Simulate web search"""
    # In production, this would call a real search API
    return {
        "query": query,
        "results": [
            {
                "title": f"Result 1 for {query}",
                "snippet": "Amazon Nova is a new generation of foundation models that deliver frontier intelligence and industry-leading price performance.",
                "url": "https://aws.amazon.com/nova/"
            },
            {
                "title": f"Result 2 for {query}",
                "snippet": "Nova models are designed for multimodal understanding, including text, images, and video.",
                "url": "https://example.com/nova-features"
            }
        ][:num_results]
    }

def save_research_note(title, content, tags=None):
    """Save research note"""
    return {
        "status": "saved",
        "note_id": f"NOTE-{datetime.now().strftime('%Y%m%d%H%M%S')}",
        "title": title,
        "tags": tags or []
    }

print("✅ Research Assistant Tools defined")

In [ ]:
# Research Assistant Agent
def run_research_assistant(research_query):
    """Run research assistant agent"""
    
    messages = [
        {
            "role": "system",
            "content": """You are a research assistant. Your role is to:
            1. Search for relevant information
            2. Analyze and synthesize findings
            3. Provide well-structured summaries
            4. Save important findings for later reference
            
            Be thorough, accurate, and cite your sources."""
        },
        {
            "role": "user",
            "content": research_query
        }
    ]
    
    print(f"\n🔬 Research Assistant Agent")
    print(f"📝 Query: {research_query}\n")
    
    # Tool dispatcher for research tools
    research_tool_funcs = {
        "web_search": web_search,
        "save_research_note": save_research_note
    }
    
    response = client.chat.completions.create(
        model=model_id,
        messages=messages,
        tools=research_tools,
        tool_choice="auto"
    )
    
    assistant_message = response.choices[0].message
    
    if assistant_message.tool_calls:
        messages.append(assistant_message)
        
        for tool_call in assistant_message.tool_calls:
            tool_name = tool_call.function.name
            tool_args = json.loads(tool_call.function.arguments)
            
            print(f"🔧 Using: {tool_name}")
            print(f"   Args: {tool_args}")
            
            tool_result = research_tool_funcs[tool_name](**tool_args)
            print(f"   Result: {json.dumps(tool_result, indent=2)}\n")
            
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": json.dumps(tool_result)
            })
        
        final_response = client.chat.completions.create(
            model=model_id,
            messages=messages
        )
        final_message = final_response.choices[0].message.content
    else:
        final_message = assistant_message.content
    
    print(f"📊 Research Summary:\n{final_message}")
    return final_message

# Test the research assistant
run_research_assistant("What are the key features of Amazon Nova AI models?")

## Use Case 3: Data Analysis Agent

This agent analyzes data and provides insights:
- Performs calculations and statistical analysis
- Generates visualizations
- Identifies trends and patterns
- Creates reports

In [ ]:
# Data Analysis Tools
data_analysis_tools = [
    {
        "type": "function",
        "function": {
            "name": "calculate_statistics",
            "description": "Calculate statistical metrics for a dataset",
            "parameters": {
                "type": "object",
                "properties": {
                    "data": {
                        "type": "array",
                        "items": {"type": "number"},
                        "description": "Array of numbers to analyze"
                    },
                    "metrics": {
                        "type": "array",
                        "items": {
                            "type": "string",
                            "enum": ["mean", "median", "std", "min", "max", "sum"]
                        },
                        "description": "Metrics to calculate"
                    }
                },
                "required": ["data", "metrics"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "analyze_trend",
            "description": "Analyze trends in time-series data",
            "parameters": {
                "type": "object",
                "properties": {
                    "data": {
                        "type": "array",
                        "items": {"type": "number"},
                        "description": "Time-series data points"
                    },
                    "period": {
                        "type": "string",
                        "description": "Time period (e.g., 'daily', 'weekly', 'monthly')"
                    }
                },
                "required": ["data"]
            }
        }
    }
]

# Tool implementations
def calculate_statistics(data, metrics):
    """Calculate statistical metrics"""
    import statistics
    results = {}
    
    if "mean" in metrics:
        results["mean"] = statistics.mean(data)
    if "median" in metrics:
        results["median"] = statistics.median(data)
    if "std" in metrics:
        results["std"] = statistics.stdev(data) if len(data) > 1 else 0
    if "min" in metrics:
        results["min"] = min(data)
    if "max" in metrics:
        results["max"] = max(data)
    if "sum" in metrics:
        results["sum"] = sum(data)
    
    return results

def analyze_trend(data, period="daily"):
    """Analyze trend in data"""
    if len(data) < 2:
        return {"trend": "insufficient_data"}
    
    # Simple trend analysis
    first_half = sum(data[:len(data)//2]) / (len(data)//2)
    second_half = sum(data[len(data)//2:]) / (len(data) - len(data)//2)
    
    change_pct = ((second_half - first_half) / first_half * 100) if first_half != 0 else 0
    
    if change_pct > 5:
        trend = "increasing"
    elif change_pct < -5:
        trend = "decreasing"
    else:
        trend = "stable"
    
    return {
        "trend": trend,
        "change_percentage": round(change_pct, 2),
        "period": period,
        "first_half_avg": round(first_half, 2),
        "second_half_avg": round(second_half, 2)
    }

print("✅ Data Analysis Tools defined")

In [ ]:
# Data Analysis Agent
def run_data_analyst(analysis_request, data=None):
    """Run data analysis agent"""
    
    # Sample data if none provided
    if data is None:
        data = [120, 135, 142, 138, 155, 168, 172, 165, 180, 195, 210, 205]
    
    messages = [
        {
            "role": "system",
            "content": f"""You are a data analyst. You have access to the following dataset:
            {data}
            
            Your role is to:
            1. Analyze data using available tools
            2. Identify patterns and trends
            3. Provide actionable insights
            4. Explain findings in clear, business-friendly language"""
        },
        {
            "role": "user",
            "content": analysis_request
        }
    ]
    
    print(f"\n📊 Data Analysis Agent")
    print(f"📝 Request: {analysis_request}")
    print(f"📈 Dataset: {data}\n")
    
    analysis_tool_funcs = {
        "calculate_statistics": calculate_statistics,
        "analyze_trend": analyze_trend
    }
    
    response = client.chat.completions.create(
        model=model_id,
        messages=messages,
        tools=data_analysis_tools,
        tool_choice="auto"
    )
    
    assistant_message = response.choices[0].message
    
    if assistant_message.tool_calls:
        messages.append(assistant_message)
        
        for tool_call in assistant_message.tool_calls:
            tool_name = tool_call.function.name
            tool_args = json.loads(tool_call.function.arguments)
            
            print(f"🔧 Analyzing with: {tool_name}")
            
            tool_result = analysis_tool_funcs[tool_name](**tool_args)
            print(f"   Results: {json.dumps(tool_result, indent=2)}\n")
            
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": json.dumps(tool_result)
            })
        
        final_response = client.chat.completions.create(
            model=model_id,
            messages=messages
        )
        final_message = final_response.choices[0].message.content
    else:
        final_message = assistant_message.content
    
    print(f"💡 Analysis:\n{final_message}")
    return final_message

# Test the data analyst
run_data_analyst("Analyze this sales data and tell me about the trend and key statistics")

## Use Case 4: Task Automation Agent

This agent automates workflows by:
- Scheduling tasks
- Sending notifications
- Managing files
- Coordinating multiple actions

In [ ]:
# Task Automation Tools
automation_tools = [
    {
        "type": "function",
        "function": {
            "name": "send_email",
            "description": "Send an email notification",
            "parameters": {
                "type": "object",
                "properties": {
                    "to": {"type": "string", "description": "Recipient email"},
                    "subject": {"type": "string", "description": "Email subject"},
                    "body": {"type": "string", "description": "Email body"}
                },
                "required": ["to", "subject", "body"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "schedule_task",
            "description": "Schedule a task for later execution",
            "parameters": {
                "type": "object",
                "properties": {
                    "task_name": {"type": "string", "description": "Name of the task"},
                    "scheduled_time": {"type": "string", "description": "When to run (ISO format)"},
                    "action": {"type": "string", "description": "Action to perform"}
                },
                "required": ["task_name", "scheduled_time", "action"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "create_file",
            "description": "Create a file with content",
            "parameters": {
                "type": "object",
                "properties": {
                    "filename": {"type": "string", "description": "Name of the file"},
                    "content": {"type": "string", "description": "File content"},
                    "format": {"type": "string", "enum": ["txt", "json", "csv", "md"]}
                },
                "required": ["filename", "content"]
            }
        }
    }
]

# Tool implementations
def send_email(to, subject, body):
    return {"status": "sent", "to": to, "subject": subject, "message_id": f"MSG-{datetime.now().strftime('%Y%m%d%H%M%S')}"}

def schedule_task(task_name, scheduled_time, action):
    return {"status": "scheduled", "task_id": f"TASK-{datetime.now().strftime('%Y%m%d%H%M%S')}", "task_name": task_name, "scheduled_time": scheduled_time}

def create_file(filename, content, format="txt"):
    return {"status": "created", "filename": f"{filename}.{format}", "size_bytes": len(content)}

print("✅ Automation Tools defined")

In [ ]:
# Task Automation Agent
def run_automation_agent(task_description):
    """Run task automation agent"""
    
    messages = [
        {
            "role": "system",
            "content": """You are a task automation assistant. You help users automate workflows by:
            1. Understanding their automation needs
            2. Breaking down complex tasks into steps
            3. Using available tools to execute tasks
            4. Providing confirmation of completed actions
            
            Be efficient and thorough."""
        },
        {
            "role": "user",
            "content": task_description
        }
    ]
    
    print(f"\n🤖 Task Automation Agent")
    print(f"📝 Task: {task_description}\n")
    
    automation_tool_funcs = {
        "send_email": send_email,
        "schedule_task": schedule_task,
        "create_file": create_file
    }
    
    response = client.chat.completions.create(
        model=model_id,
        messages=messages,
        tools=automation_tools,
        tool_choice="auto"
    )
    
    assistant_message = response.choices[0].message
    
    if assistant_message.tool_calls:
        messages.append(assistant_message)
        
        for tool_call in assistant_message.tool_calls:
            tool_name = tool_call.function.name
            tool_args = json.loads(tool_call.function.arguments)
            
            print(f"⚙️ Executing: {tool_name}")
            print(f"   Parameters: {tool_args}")
            
            tool_result = automation_tool_funcs[tool_name](**tool_args)
            print(f"   ✅ Result: {tool_result}\n")
            
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": json.dumps(tool_result)
            })
        
        final_response = client.chat.completions.create(
            model=model_id,
            messages=messages
        )
        final_message = final_response.choices[0].message.content
    else:
        final_message = assistant_message.content
    
    print(f"📋 Summary:\n{final_message}")
    return final_message

# Test automation agent
run_automation_agent("Send a summary email to team@company.com about today's meeting and create a meeting notes file")

## Use Case 5: Multi-Agent System

Coordinate multiple specialized agents to solve complex problems:
- Router agent decides which specialist to use
- Specialist agents handle specific domains
- Aggregator combines results

In [ ]:
# Multi-Agent System
class MultiAgentSystem:
    """Coordinate multiple specialized agents"""
    
    def __init__(self):
        self.agents = {
            "customer_support": {
                "description": "Handles customer service inquiries, orders, and support tickets",
                "tools": customer_support_tools
            },
            "research": {
                "description": "Conducts research, searches information, and provides summaries",
                "tools": research_tools
            },
            "data_analysis": {
                "description": "Analyzes data, calculates statistics, and identifies trends",
                "tools": data_analysis_tools
            },
            "automation": {
                "description": "Automates tasks like sending emails, scheduling, and file management",
                "tools": automation_tools
            }
        }
    
    def route_request(self, user_request):
        """Determine which agent should handle the request"""
        
        # Create agent descriptions for routing
        agent_list = "\n".join([
            f"- {name}: {info['description']}"
            for name, info in self.agents.items()
        ])
        
        routing_prompt = f"""Given the following user request, determine which specialized agent should handle it.
        
Available agents:
{agent_list}

User request: {user_request}

Respond with ONLY the agent name (customer_support, research, data_analysis, or automation)."""
        
        response = client.chat.completions.create(
            model=model_id,
            messages=[{"role": "user", "content": routing_prompt}],
            temperature=0
        )
        
        selected_agent = response.choices[0].message.content.strip().lower()
        
        # Validate agent selection
        if selected_agent not in self.agents:
            # Default to customer support if unclear
            selected_agent = "customer_support"
        
        return selected_agent
    
    def execute(self, user_request):
        """Route and execute request through appropriate agent"""
        
        print(f"\n🎯 Multi-Agent System")
        print(f"📝 Request: {user_request}\n")
        
        # Route to appropriate agent
        selected_agent = self.route_request(user_request)
        print(f"🔀 Routing to: {selected_agent.upper()} agent\n")
        
        # Execute with selected agent
        if selected_agent == "customer_support":
            return run_customer_support_agent(user_request)
        elif selected_agent == "research":
            return run_research_assistant(user_request)
        elif selected_agent == "data_analysis":
            return run_data_analyst(user_request)
        elif selected_agent == "automation":
            return run_automation_agent(user_request)

# Create multi-agent system
mas = MultiAgentSystem()
print("✅ Multi-Agent System initialized")

In [ ]:
# Test 1: Customer support query
mas.execute("I need to check my order status for ORD-99999")

In [ ]:
# Test 2: Research query
mas.execute("Find information about AI agent frameworks")

In [ ]:
# Test 3: Automation task
mas.execute("Schedule a reminder for tomorrow at 9am to review the quarterly report")

## Bonus: Building Agents with Strands SDK

Strands SDK provides a higher-level abstraction for building agents with Amazon Nova.
It handles conversation state, tool orchestration, and more.

In [ ]:
# Strands SDK Example
from strands import Agent
from strands_tools import http_request

# Set bypass consent for demo purposes
os.environ["BYPASS_TOOL_CONSENT"] = "true"

# Create a Strands agent with HTTP tool
strands_agent = Agent(
    name="WebResearchAgent",
    tools=[http_request],
    model=model_id,
    api_key=api_key,
    base_url=base_url
)

print("✅ Strands Agent created")
print(f"   Name: {strands_agent.name}")
print(f"   Tools: {[tool.__name__ if hasattr(tool, '__name__') else str(tool) for tool in strands_agent.tools]}")

In [ ]:
# Run Strands agent
# Note: This is a simplified example. See the strands/ directory for more advanced examples
print("\n🌟 Strands SDK provides:")
print("  - Automatic conversation state management")
print("  - Built-in tool orchestration")
print("  - Memory and context handling")
print("  - Pre-built community tools")
print("\nFor detailed Strands examples, check out:")
print("  - strands/strands-nova-quickstart.ipynb")
print("  - strands/runtime_with_strands_and_nova_model.ipynb")

## Best Practices for Building AI Agents

### 1. **Design Clear Tool Interfaces**
- Use descriptive function names
- Provide detailed parameter descriptions
- Include examples in descriptions
- Use enums for constrained choices

### 2. **Handle Errors Gracefully**
- Validate tool inputs
- Return structured error messages
- Implement retry logic for transient failures
- Log errors for debugging

### 3. **Optimize for Performance**
- Use `nova-2-lite-v1` for simple tasks (faster, cheaper)
- Use `nova-2-pro-v1` for complex reasoning
- Cache frequently used results
- Minimize tool calls when possible

### 4. **Maintain Context**
- Keep conversation history for multi-turn interactions
- Use system prompts to set agent behavior
- Clear context when starting new tasks

### 5. **Security Considerations**
- Validate all user inputs
- Implement rate limiting
- Use environment variables for API keys
- Never expose sensitive data in prompts
- Implement user consent for sensitive operations

### 6. **Testing and Monitoring**
- Test agents with edge cases
- Monitor token usage and costs
- Track agent performance metrics
- Implement logging for debugging

## Next Steps

### Explore More Examples:
1. **Basic Tutorials**
   - `00_getting_started.ipynb` - Nova API basics
   - `01_multi_turn_interactions.ipynb` - Conversations
   - `02_multimodal_understanding.ipynb` - Images & video
   - `03_tool_use_with_nova.ipynb` - Tool calling

2. **Advanced Agent Examples**
   - `langchain/` - LangChain integration patterns
   - `strands/` - Strands SDK advanced examples
   - `nova_agents/` - Computer use with Nova Act

### Build Your Own Agent:
1. Identify your use case
2. Define the tools your agent needs
3. Create tool implementations
4. Design the agent's system prompt
5. Test with various inputs
6. Iterate and improve

### Resources:
- [Amazon Nova Documentation](https://nova.amazon.com/dev/documentation)
- [API Reference](https://nova.amazon.com/dev/api)
- [AWS Nova Samples](https://github.com/aws-samples/amazon-nova-samples/)

---

**Happy Building! 🚀**